In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import bambi as bmb
import seaborn as sns
from pygam.datasets import mcycle
from pygam import LinearGAM, s, f, GAM, l

In [ ]:
# url for the data is: 
url = "https://raw.githubusercontent.com/m-clark/generalized-additive-models/master/data/pisasci2006.csv"
pisa = pd.read_csv(url)
pisa

## Variation due GAM penalisation and parameterisation


In [ ]:
X, y = mcycle(return_X_y=True)

gam1 = LinearGAM(s(0, n_splines=5)).fit(X, y)
gam2 = LinearGAM(s(0, n_splines=10)).fit(X, y)
gam3 = LinearGAM(s(0, n_splines=15)).fit(X, y)
gam4 = LinearGAM(s(0, lam=.1)).fit(X, y)
gam5 = LinearGAM(s(0, lam=.3)).fit(X, y)
gam6 = LinearGAM(s(0, lam=.6)).fit(X, y)


def plot_fit(gam, X, y, ax, t, c1='b', c2='r'):
    XX = gam.generate_X_grid(term=0, n=500)

    ax.plot(XX, gam.predict(XX), color=c2, linestyle='--')
    ax.plot(XX, gam.prediction_intervals(XX, width=.95), color=c1, ls='--')

    ax.scatter(X, y, facecolor='gray', edgecolors='none')
    ax.set_title(f'95% prediction interval with {t}');

fig, axs = plt.subplots(3,2, figsize=(10, 15))
axs = axs.flatten()
titles = ['5_splines', '10_splines', '15_splines', 
'lam=.1', 'lam=.3', 'lam=.6']
gs = [gam1, gam2, gam3, gam4, gam5, gam6]
for ax, g, t in zip(axs, gs, titles):
    plot_fit(g, X, y, ax, t)


## Optimising The Parameter Setting


In [ ]:
## model raw
gam_raw = LinearGAM(s(0)).fit(X, y)
gam_raw.summary()

In [ ]:
## model optimised
gam = LinearGAM(s(0))
gam.gridsearch(X, y)
gam.summary()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plot_fit(gam_raw, X, y, ax, "Unoptimised Fit", c1='orange', c2='green')
plot_fit(gam, X, y, ax, "Optimised Fit")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plot_fit(gam, X, y, ax, "Optimised Fit")

In [ ]:
plt.figure(figsize=(10,20), dpi= 80)
g = sns.pairplot(pisa[['Overall', 'Health', 'Support', 'Edu']], kind="reg", diag_kind='kde')
g.fig.suptitle("Pair Plot of Score Metrics", y=1.08, fontsize=20)

In [ ]:
model_df = pisa[['Overall', 'Edu', 'Health', 'Support', 'Income']].dropna()
X = model_df[['Income', 'Edu', 'Health']].values
y = model_df['Overall'].values
gam = LinearGAM(s(0, n_splines=20) + s(1, n_splines=15) +s(2, n_splines=10)).fit(X, y)
gam.gridsearch(X, y)


gam.summary()

In [ ]:
import statsmodels.api as sm

sm.OLS(y, sm.add_constant(X)).fit().summary()


In [ ]:
fig, axs = plt.subplots(1,3);
titles = ['Edu', 'Health', 'Support']
for i, ax in enumerate(axs):
    XX = gam.generate_X_grid(term=i)
    ax.plot(XX[:, i], gam.partial_dependence(term=i, X=XX))
    ax.plot(XX[:, i], gam.partial_dependence(term=i, X=XX, width=.95)[1], c='r', ls='--')
    if i == 0:
        ax.set_ylim(-30,70)
    ax.set_title(titles[i]);
